### Imports

In [9]:
import sys
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_files
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
from twitter import Twitter
from twitter import oauth_dance, OAuth
import pandas

In [2]:
CONSUMER_KEY = 'QdrOrGTx0JqAzTSAZkeF8ZCxT'
CONSUMER_SECRET = '5TNSrWNfOq2ek55HMAjKbgmd24NSUY8fbWhmZeHNyQ6ELVEl4b'

### Twitter related functions

In [13]:
def dance_with_twitter():
    # 'QdrOrGTx0JqAzTSAZkeF8ZCxT',
    # '5TNSrWNfOq2ek55HMAjKbgmd24NSUY8fbWhmZeHNyQ6ELVEl4b',
    # '1606571954-nQrQ4iQaOku7NcABsB44y1bp1ukSkYGumZNMeE8',
    # 'dzMLIjhcnABo3fCJITbyOhoNh84eOM6Kh289bilVxSXx2'
    oauth_token, oauth_token_secret = oauth_dance("Test Bot", CONSUMER_KEY, CONSUMER_SECRET)
    return oauth_token, oauth_token_secret

In [14]:
def connect_to_twiter(oauth_token, oauth_token_secret):
    try:
        t = Twitter(auth=OAuth(oauth_token, oauth_token_secret, CONSUMER_KEY, CONSUMER_SECRET))
    except:
        print("can't connect to twitter. bummer. frowny face.")

    return t

In [15]:
def query_twitter(t, subject):
    tweets = t.search.tweets(q='#' + subject)
    return tweets

### Test Twitter API

In [16]:
oauth_token, oauth_token_secret = dance_with_twitter()

Hi there! We're gonna get you all set up to use Test Bot.

In the web browser window that opens please choose to Allow
access. Copy the PIN number that appears on the next page and paste or
type it here:

Opening: https://api.twitter.com/oauth/authorize?oauth_token=DpQuugAAAAAAiuD4AAABUzHF0WI

Please enter the PIN: 6989830


In [18]:
t = connect_to_twiter(oauth_token, oauth_token_secret)

In [45]:
tweets = t.search.tweets(q='#koala #culling')

In [43]:
len(tweets['statuses'])
for i in range(len(tweets['statuses'])):
    print tweets['statuses'][i]['text']

0

# Data from other Twitter scraper

In [3]:
tweets = pandas.read_csv('cull.csv', sep=';')

In [4]:
print tweets['text'][10]

Caught between a rock & a hard place, #Australia needs to cull overabundant" #koala colony to save #environment : http:// ow.ly/3xVtW5"


# Sentiment analysis training

In [5]:
def train_with_data(folder):
    movie_reviews_data_folder = folder
    dataset = load_files(movie_reviews_data_folder, shuffle=False)
    print("n_samples: %d" % len(dataset.data))

    # split the dataset in training and test set:
    docs_train, docs_test, y_train, y_test = train_test_split(
        dataset.data, dataset.target, test_size=0.25, random_state=None)

    # TASK: Build a vectorizer / classifier pipeline that filters out tokens
    # that are too rare or too frequent
    pipeline = Pipeline([
        ('vect', TfidfVectorizer(min_df=3, max_df=0.95)),
        ('clf', LinearSVC(C=1000)),
    ])

    # TASK: Build a grid search to find out whether unigrams or bigrams are
    # more useful.
    # Fit the pipeline on the training set using grid search for the parameters
    parameters = {
        'vect__ngram_range': [(1, 1), (1, 2)],
    }
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1)
    grid_search = grid_search.fit(docs_train, y_train)
    
    # TASK: print the cross-validated scores for the each parameters set
    # explored by the grid search
    print(grid_search.grid_scores_)

    # TASK: Predict the outcome on the testing set and store it in a variable
    # named y_predicted
    y_predicted = grid_search.predict(docs_test)

    # Print the classification report
    print(metrics.classification_report(y_test, y_predicted,
                                        target_names=dataset.target_names))

    # Print and plot the confusion matrix
    cm = metrics.confusion_matrix(y_test, y_predicted)
    print(cm)

    #plt.matshow(cm)
    #plt.show()
    
    return dataset, grid_search

In [6]:
dataset, grid_search = train_with_data('txt_sentoken/')

n_samples: 2000
[mean: 0.82867, std: 0.01789, params: {'vect__ngram_range': (1, 1)}, mean: 0.84467, std: 0.01088, params: {'vect__ngram_range': (1, 2)}]
             precision    recall  f1-score   support

        neg       0.85      0.89      0.87       243
        pos       0.89      0.86      0.87       257

avg / total       0.87      0.87      0.87       500

[[217  26]
 [ 37 220]]


In [7]:
with open("sentiment_analysis_of_koala_cull_based_on_movies_training_set.csv", 'w+') as f:
    for tweet in tweets['text']:
        f.write(dataset.target_names[grid_search.predict([str(tweet)])] + ';' + str(tweet) + "\n")
        print(dataset.target_names[grid_search.predict([str(tweet)])], str(tweet))

/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:3: DeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  app.launch_new_instance()
/usr/local/lib/python2.7/site-packages/ipykernel/__main__.py:4: DeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future


('neg', 'Koala cull suggested to save native marsupials from chlamydia http:// ift.tt/1R8ggaP')
('neg', "Hey guys, there's a koala cull in Australia . http://www. abc.net.au/news/2015-10-0 6/health-of-great-ocean-road-koala-population-improving/6829156 ")
('pos', "Australia begins koala cull http:// a.msn.com/01/en-au/AAeSV Ld?ocid=st Witness the truly phenomenal problem-solving abilities of the 'great' human brain.")
('neg', 'Australia begins koala cull to save species from mass starvation http:// dlvr.it/CHjhK2 pic.twitter.com/3GX9WJ4Zg3')
('neg', 'Koala overpopulation forces Australia to start largest cull , relocation plan http:// xhne.ws/Jwefc (AFP pics) pic.twitter.com/Tsqn159wlh')
('pos', 'Australia mulls koala cull , Japans whaling advocates eat up the irony like delicious whalemeat http:// en.rocketnews24.com/2015/05/28/aus tralia-mulls-koala-cull-japans-whaling-advocates-eat-up-the-irony-like-delicious-whale-meat/ via @RocketNews24En')
('neg', 'Koala cull underway at Cape Otw

In [13]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(tweets['text'])
X_train_counts.shape
count_vect.vocabulary_.get(u'algorithm')
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

In [14]:
X_train_tfidf.shape

(57, 470)

In [16]:
with open("take2.csv", 'w+') as f:
    for tweet in tweets['text']:
        f.write(dataset.target_names[grid_search.predict(X_train_tfidf)] + ';' + str(tweet) + "\n")
        print(dataset.target_names[grid_search.predict([str(tweet)])], str(tweet))

AttributeError: lower not found

In [22]:
X_train_tfidf

  (0, 46)	0.259096428837
  (0, 233)	0.259096428837
  (0, 328)	0.259096428837
  (0, 343)	0.259096428837
  (0, 298)	0.235041598201
  (0, 203)	0.217974424253
  (0, 300)	0.122492086491
  (0, 212)	0.259096428837
  (0, 6)	0.259096428837
  (0, 33)	0.193919593617
  (0, 295)	0.163614091075
  (0, 86)	0.184774371447
  (0, 294)	0.259096428837
  (0, 56)	0.259096428837
  (0, 460)	0.152797589032
  (0, 91)	0.0603583035194
  (0, 234)	0.0892920336846
  (0, 399)	0.235041598201
  (0, 205)	0.259096428837
  (0, 213)	0.259096428837
  (0, 219)	0.0669569325049
  (0, 146)	0.0603583035194
  (0, 257)	0.11865302417
